# Network Simulation: Hardy Cross Method

## Definitions
- $N_j$, the number of junction nodes: nj
- $N_p$, the number of pipes: np
- $N_l$, the number of independent loops: nl
- $h$, an array storing headlosses of pipes: HL
- $H$, an array storing nodal heads: H
- $i$, the row number of a node: i
- $j$, the column number of a node: j
- $r$, total rows of the network: r
- $c$, total columns of the network: c
- $

Let's start from the simpliest network: a rectangle divided by one vertical pipe in the middle to make two squares side by side, i.e. two independent loops. This 2 rows and 3 columns setup makes the network have 6 (rxc) nodes, which can be denoted by a 2x3 array N. The network has 7 pipes, which can be divided into 4 (rx(c-1)) horizontal pipes and 3 ((r-1)xc) vertical pipes, which can be denoted by a 2x2 array PH and a 1x3 array PV, respectively. These numbers can be generarized if r and c are given. 


In [1]:
import numpy as np

r = 2 # total rows of the network
c = 3 # total columns of the network

# Create arrays
NH = np.zeros((r,c)) # Node Head array
NQ = np.zeros((r,c)) # Node flowrate array: negative if there is discharged water from the network at the node; positive if there is charged water into the network at the node; zero if just water transportation at the node. 

HPL = np.ones((r,c-1)) # horizontal pipe length array
HPD = np.ones((r,c-1)) # horizontal pipe diameter array
HPQ = np.zeros((r,c-1)) # horizontal pipe flowrate array: positive value if to the right
HPK = np.ones((r,c-1)) # horizontal pipe coefficient array
HPH = np.ones((r,c-1)) # horizontal pipe headloss array

VPL = np.ones((r-1,c)) # vertical pipe length array
VPD = np.ones((r-1,c)) # vertical pipe diameter array: positive value if downward
VPQ = np.zeros((r-1,c)) # vertical pipe flowrate array
VPK = np.zeros((r-1,c)) # vertical pipe coefficient array
VPH = np.zeros((r-1,c)) # vertical pipe headloss array

IP = np.ones((r-1,c-1)) # independent pipe loop array storing deltaQ, DQ: positive if counterclockwise

In [2]:
# Assign values to arrays based on the given information of the problem
HPL[:]=5000
VPL[:]=4000
HPD[:]=[24,12],[18,18]
VPD[:]=[18,18,18]
NQ[:]=[10,0,-2],[0,-4,-4]

# constant
f = 0.02 # Darcy-Weisbach friction factor

# Calculate K values and update the values in the HPK and VPK arrays
#HPK=8*f*HPL/(3.14**2*9.81*(HPD/12)**5) I will go with the simplified equation in the textbook as follows
HPK = HPL/(1959*(HPD/12)**5)
VPK = VPL/(1959*(VPD/12)**5)


In [4]:
# Manually initialize the flow rate in each pipe, i.e. change values of HPQ and VPQ arrays
HPQ[:]=[6.2,2.2],[3.8,3.8]
VPQ[:]=[3.8,4.0,0.2]
HPQ

array([[6.2, 2.2],
       [3.8, 3.8]])

In [5]:
m = 1
while True:
    if np.abs(IP).sum()>0.01:
        print('Loop # ', m)
        for i in np.arange(r-1):
            for j in np.arange(c-1):
                #update IP values with delta Q calculations:
                IP[i,j]=(-1/2)*(-HPK[i,j]*HPQ[i,j]**2+HPK[i+1,j]*HPQ[i+1,j]**2+VPK[i,j]*VPQ[i,j]**2-VPK[i,j+1]*VPQ[i,j+1]**2)/(HPK[i,j]*np.abs(HPQ[i,j])+HPK[i+1,j]*np.abs(HPQ[i+1,j])+VPK[i,j]*np.abs(VPQ[i,j])+VPK[i,j+1]*np.abs(VPQ[i,j+1]))
        print(IP)
        # Adjust horizontal pipe flows based on the delta Q for each loop.
        for j in np.arange(c-1):
            k=0
            for i in np.arange(r):
                if k==0:
                    HPQ[i,j]=HPQ[i,j]-IP[0,j]
                else:
                    HPQ[i,j]=HPQ[i,j]+IP[0,j]
                k+=1
                k=k%2
        print(HPQ)
        # Adjust vertical pipe flows. 
        for i in np.arange(r-1):
            for j in np.arange(0,c,3):
                VPQ[i,j]=VPQ[i,j]+IP[0,j]
                VPQ[i,j+1]=VPQ[i,j+1]-IP[0,j]+IP[0,j+1]
                VPQ[i,j+2]=VPQ[i,j+2]-IP[0,j+1]
        print(VPQ)
        m+=1
    else:
        break

Loop #  1
[[-0.17678332  0.19998429]]
[[6.37678332 2.00001571]
 [3.62321668 3.99998429]]
[[3.62321668e+00 4.37676761e+00 1.57125648e-05]]
Loop #  2
[[ 0.05827725 -0.0209192 ]]
[[6.31850607 2.02093491]
 [3.68149393 3.97906509]]
[[3.68149393 4.29757115 0.02093491]]
Loop #  3
[[-0.00634647  0.00891829]]
[[6.32485254 2.01201662]
 [3.67514746 3.98798338]]
[[3.67514746 4.31283592 0.01201662]]
Loop #  4
[[ 0.00265625 -0.00095125]]
[[6.32219629 2.01296787]
 [3.67780371 3.98703213]]
[[3.67780371 4.30922842 0.01296787]]
